# Process Weekly Calendar Dates File

## Overview

A `calendar_dates.txt` file is being generated on a weekly basis.  It contains the data for the upcoming two weeks.  Each new `calendar_dates.txt` overlaps the previous one by 1 week, meaning each new file can't just be appended to the main file.  The rows cannot just be added and subtracted to determine what the final row count should be.

> As a temporary measure, data consumers will need to merge this weekly file with the original shakeup file by removing the data for the upcoming two week period from the original shakeup file, and then adding the data from the new file.

Automation Steps:

1. Load the original `calendar_dates.txt` into a dataframe

2. For each zip file:

    1. Make a new directory based on its filename and extract the contents to that directory
    2. Parse the filename to get a date range
    3. Remove that date range from the original `calendar_dates.txt`
    4. Load the new `calendar_dates.txt` into a dataframe
    5. Append the new `calendar_dates.txt` to the original `calendar_dates.txt`
    6. Repeat

3. Clean up the original dataframe that we've now modified and export it to replace the original file in the GTFS.

### Notes

Use `os.path.join` with the `DIR` constants to build the correct paths.

### Process for Updates

Run this notebook.  Update README.md with the details.


In [10]:
import pandas as pd
import os
import datetime as dt
from zipfile import ZipFile

ROOT_DIR = os.path.normpath("../")
INPUT_DIR = os.path.join(ROOT_DIR, 'notebooks/input')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'calendar_dates')
SCRATCH_DIR = os.path.join(ROOT_DIR, 'notebooks/scratch')
CALENDAR_DATES_FILE = os.path.join(ROOT_DIR, 'calendar_dates.txt')

# create Output and Scratch directories if they don't already exist.
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SCRATCH_DIR, exist_ok=True)

## Load Original `calendar_dates.txt` Data

Create a new column and populated it with values from the `date` column after convertin to the `Date` type.


In [2]:
shakeup_cal_df = pd.read_csv(CALENDAR_DATES_FILE, dtype= {'service_id':'str', 'date':'str', 'exception_type': 'Int64'})

def convert_date (row):
	year = row.date[0:4]
	month = row.date[4:6]
	day = row.date[6:8]

	return dt.date(int(year), int(month), int(day))
	
shakeup_cal_df['converted_date'] = shakeup_cal_df.apply(lambda row: convert_date(row), axis=1)

print(shakeup_cal_df)

                           service_id      date  exception_type converted_date
0     SEPT21-D02CAR-1_Weekday-SANTHA0  20211213               2     2021-12-13
1     SEPT21-D02CAR-1_Weekday-SANTHA0  20211214               2     2021-12-14
2     SEPT21-D02CAR-1_Weekday-SANTHA0  20211215               2     2021-12-15
3     SEPT21-D02CAR-1_Weekday-SANTHA0  20211216               2     2021-12-16
4     SEPT21-D02CAR-1_Weekday-SANTHA0  20211217               2     2021-12-17
...                               ...       ...             ...            ...
3389        SoFi_Stadium_Express_1305  20211212               1     2021-12-12
3390        SoFi_Stadium_Express_1720  20211216               1     2021-12-16
3391        SoFi_Stadium_Express_1305  20211219               1     2021-12-19
3392        SoFi_Stadium_Express_1305  20220102               1     2022-01-02
3393        SoFi_Stadium_Express_1305  20220109               1     2022-01-09

[3394 rows x 4 columns]


## Replace `calendar_dates.txt` Data

In [3]:
# Loop through the zip files in order from earliest to newest
for root, dirs, files in os.walk(INPUT_DIR):
	# Use sorted to make sure files get returned with the earliest one first
	for file_name in sorted(files): 
		file_path = os.path.join(INPUT_DIR, file_name)
		dates = file_name.replace(' ', '').split('calendar_dates')[1].split('.zip')[0].split('-')
		dates = dates[0:2]
		print (dates)
		print('File: ' + file_name)

		# Get the date range of this zip file
		date_range = []
		for d in dates:
			year = '20' + d[0:2]
			month = d[2:4]
			day = d[4:6]

			parsed_date = dt.date(int(year), int(month), int(day))
			date_range.append(parsed_date)
		print('Date Range: ' + str(date_range))
		
		# Make a new directory and extract the contents to that directory
		with ZipFile(file_path, 'r') as zf:
			new_dir = os.path.join(SCRATCH_DIR, file_name.split('.')[0])
			os.makedirs(new_dir, exist_ok=True)
			zf.extractall(new_dir)

		# Remove the rows that match the date range of the zip file
		print('Starting rows: ' + str(shakeup_cal_df.shape[0]))
		print('Number of rows to remove: ' + str(shakeup_cal_df[(shakeup_cal_df.converted_date >= date_range[0]) & (shakeup_cal_df.converted_date <= date_range[1])].shape[0]))
		shakeup_cal_df.drop(shakeup_cal_df[(shakeup_cal_df.converted_date >= date_range[0]) & (shakeup_cal_df.converted_date <= date_range[1])].index, inplace=True)

		# Load the new data
		new_cal_df = pd.read_csv(file_path, 
			dtype= {'service_id':'str', 'date':'str', 'exception_type': 'Int64'})
		print('Number of rows to add: ' + str(new_cal_df.shape[0]))

		# Append the new data
		shakeup_cal_df = shakeup_cal_df.append(new_cal_df, ignore_index=True)

		# Re-convert the dates for the new data
		shakeup_cal_df['converted_date'] = shakeup_cal_df.apply(lambda row: convert_date(row), axis=1)

		print('Final number of rows after adding new data: ' + str(shakeup_cal_df.shape[0]))

# Clean up the original dataframe and export it
shakeup_cal_df.drop(columns='converted_date', inplace=True)
shakeup_cal_df = shakeup_cal_df.sort_values(by=['date', 'service_id'])
shakeup_cal_df.to_csv(os.path.join(OUTPUT_DIR, 'calendar_dates.txt'), index=False)

['210922', '211006']
File: 210922 - biweekly calendar_dates 210922 - 211006.zip
Date Range: [datetime.date(2021, 9, 22), datetime.date(2021, 10, 6)]
Starting rows: 3394
Number of rows to remove: 504
Number of rows to add: 181
Final number of rows after adding new data: 3071
['210929', '211013']
File: 210929 - weekly calendar_dates 210929 - 211013.zip
Date Range: [datetime.date(2021, 9, 29), datetime.date(2021, 10, 13)]
Starting rows: 3071
Number of rows to remove: 324
Number of rows to add: 168
Final number of rows after adding new data: 2915
['211006', '211020']
File: 211006 - weekly calendar_dates 211006 - 211020.zip
Date Range: [datetime.date(2021, 10, 6), datetime.date(2021, 10, 20)]
Starting rows: 2915
Number of rows to remove: 311
Number of rows to add: 193
Final number of rows after adding new data: 2797
['211013', '211027']
File: 211014 - weekly calendar_dates 211013 - 211027.zip
Date Range: [datetime.date(2021, 10, 13), datetime.date(2021, 10, 27)]
Starting rows: 2797
Number o

In [16]:
import time
import pandas as pd 
import os
import datetime as dt
from zipfile import ZipFile

ROOT_DIR = os.path.normpath("../")
# INPUT_DIR = os.path.join(ROOT_DIR, 'notebooks/input')
# OUTPUT_DIR = os.path.join(ROOT_DIR, 'calendar_dates')
# SCRATCH_DIR = os.path.join(ROOT_DIR, 'notebooks/scratch')
CALENDAR_DATES_FILE = os.path.join("scratch/", 'calendar_dates.txt')

# create Output and Scratch directories if they don't already exist.
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# os.makedirs(SCRATCH_DIR, exist_ok=True)
start = time.perf_counter()

shakeup_cal_df = pd.read_csv(CALENDAR_DATES_FILE, dtype= {'service_id':'str', 'date':'str', 'exception_type': 'Int64'})

def convert_date (row):
	year = row.date[0:4]
	month = row.date[4:6]
	day = row.date[6:8]

	return dt.date(int(year), int(month), int(day))
	
shakeup_cal_df['converted_date'] = shakeup_cal_df.apply(lambda row: convert_date(row), axis=1)

end = time.perf_counter()
total_time = round(end - start, 2)
print(str(total_time)+ " seconds")

0.78seconds


In [49]:
import datetime

def convert_text_to_date(date_text):
	year = date_text[0:4]
	month = date_text[4:6]
	day = date_text[6:8]
	return datetime.date(int(year), int(month), int(day))

# start = time.perf_counter()

with (open(CALENDAR_DATES_FILE, 'r')) as f:
	next(f, None) # skip the header
	lines = f.readlines()
	final_calendar_dates = []
	for line in lines:
		line_data = line.split(',')
		line_data[1] = str(convert_text_to_date(line_data[1]))
		final_calendar_dates.append(line_data)


# end = time.perf_counter()
# total_time = round(end - start, 2)
# print(str(total_time)+ " seconds")

0.08 seconds


## Temporary Extraction Code

Use this to put the new `calendar_dates.txt` file into the `calendar_dates/` directory as a temporary solution while the final automation code is being worked on.

In [4]:
# for file_path in file_list:
#     with ZipFile(file_path, 'r') as zf:
#         zf.extractall('../calendar_dates/')